# QBE

## Goal
Modernize the way we deliver models and technical premium to our business partners (e.g.,
Underwriters, Claims Reps, etc.) Our aim is to deploy these models as an API, such that they can
be easily consumed from various downstream applications. In addition to this, we’re interested
to see a solution with the following components:
- Deployment Governance (e.g., approval process to move to production, version history,
etc.)
- Complete, searchable, and auditable provenance of all models
- Model explainability (e.g., which features were important to the prediction and why)
- Model quality monitoring (e.g., model accuracy over time)
- Data quality monitoring (e.g., data drift)
- Holistic view of all models in production (e.g., how many models do we have, what’s the aggregate model quality, how many need to be retrained, etc.)
- API telemetry and transaction auditability (e.g., how often is the API used, how performant is it, what were the inputs and outputs in each transaction, etc.)
- Native support for development in open-source languages (e.g., R, Python, etc.) and deployment of varying model types to the solution (e.g., GLM, GBM, CNN, etc.)
- Ability to make pricing transactional data available for analytical purposes

## Load Libraries

In [1]:
!pip3 install --upgrade pip
!pip3 install --user -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


## Import Dependencies

In [3]:
import requests
import numpy as np 
import pandas as pd
import scipy
import statsmodels.api as sm

## Load Data 

In [40]:
data = pd.read_csv("comm_auto_sample_data.csv")
data.info()
indication = pd.read_csv("comm_auto_sample_indication.csv")
indication.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131773 entries, 0 to 131772
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   coverage_type                    131773 non-null  object 
 1   pol_eff_year                     131773 non-null  int64  
 2   risk_state                       131773 non-null  object 
 3   naics3                           131539 non-null  float64
 4   n_power_units                    131773 non-null  int64  
 5   prior_claim_freq_3yr             99408 non-null   float64
 6   dnb_credit_score                 108504 non-null  float64
 7   easi_snowfall                    131504 non-null  float64
 8   experience_rated_manual_premium  131773 non-null  float64
 9   incurred_loss_and_alae           131773 non-null  float64
 10  split                            131773 non-null  object 
dtypes: float64(6), int64(2), object(3)
memory usage: 11.1+ MB
<class 

## Split Data

In [91]:
train_df = data[data.split == 'Training']
train_df = train_df.drop(['split', 'coverage_type', 'risk_state'], axis=1)
train_df = train_df.fillna(0)
train_x = sm.add_constant(train_df)
train_x.head()

train_y = pd.DataFrame(train_df["incurred_loss_and_alae"] / train_df["experience_rated_manual_premium"])
train_y = train_y.rename(columns={0: "manual_loss_ratio",})
train_y.head()

test_df = data[data.split == 'Holdout']
test_df = test_df.drop(['split', 'coverage_type'], axis=1)
test_df = test_df.fillna(0)
test_df = sm.add_constant(test_df)
test_df.head()

,const,pol_eff_year,risk_state,naics3,n_power_units,prior_claim_freq_3yr,dnb_credit_score,easi_snowfall,experience_rated_manual_premium,incurred_loss_and_alae
0,1.0,2010,AK,722.0,24,0.0,0.0,70.0,478.555077,0.000000
8,1.0,2010,AR,453.0,1,0.0,2.0,5.0,429.164457,0.000000
9,1.0,2010,AR,423.0,4,0.0,1.0,7.0,1234.672690,0.000000
18,1.0,2010,AZ,112.0,17,0.0,3.0,0.0,3274.963002,423.294916
19,1.0,2010,AZ,112.0,32,0.0,3.0,0.0,2978.807420,0.000000


## Sample Use Case
To deliver technical premium for Commercial Retail – Auto, you need a combination of pricing
inputs from:
### 1. Manual Premium
- Comes from our Policy Admin System, Majesco
- Would be API data from Majesco
- For the demo, briefly describe how you would ingest this type of data from an internal vendor.

In [9]:
url = "https://httpbin.org/post"
querystring = {"search.sessiontype":"1522435540042001BxTD"}
payload = {}
headers = {'content-type': "multipart/form-data"}
print(requests.request("POST", url, data=payload, headers=headers, params=querystring).text)

{
  "args": {
    "search.sessiontype": "1522435540042001BxTD"
  }, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "0", 
    "Content-Type": "multipart/form-data", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.25.1", 
    "X-Amzn-Trace-Id": "Root=1-61bcc5ac-06e750c523b95aa03aaba88e"
  }, 
  "json": null, 
  "origin": "35.226.118.171", 
  "url": "https://httpbin.org/post?search.sessiontype=1522435540042001BxTD"
}



### 2. Experience Mod (tool_name = Experience Rater, file_type = Excel, model_type = Rater)
- Relies on historical losses, which comes in the form of PDFs, Excel files, etc.
- Sample Algorithm:

`If Manual Premium < $10,000
    If losses > $20,000 then 2 else 0.8
Else
    If losses > $1,000,000 then 1.5 else 1.0`

In [74]:
def experience_rater(manual_premium, losses):
    rating = 0.0
    if manual_premium < 10000:
        if losses > 20000:
            rating = 2.0
        else:
            rating = 0.8
    else:
        if losses > 1000000:
            rating = 1.5
        else:
            rating = 1
    return rating

In [75]:
print(experience_rater(20000,0))

1


### 3. Loss Rating Tool (tool_name = CLRT, file_type = Excel, model_type = Rater)
- Relies on (1) and (2)
- Only used for large accounts
    - $500,000+ experience rated manual premium
    - This is an example of a business rule we’d like to see handled (i.e., when to use a specific algorithm)
- In production, the input to this algorithm is dynamic (e.g., what if analysis), and will come from our Underwriting Workbench (i.e., the UI where the Underwriter works to quote policies).
- Example Input Variables
    - Historical exposure by year
        - Auto exposure is vehicle count
    - Losses by year
    - Claim detail
    - Aggregate losses by year
    - Account information
- Please describe how you would handle the dynamic input and output of Example Input Variables as part of your deployment
- Output is technical premium
    - CLRT Mod * Experience Mod * Manual Premium = Technical Premium
    - CLRT Mod Sample Algorithm:
    
`If exposure < 100
    If losses > 1000000 then 1.8
    If losses > 500000 then 1.4
    If losses > 250000 then 1.0
    If losses > 100000 then 0.9
    Else 0.8
Else
    If losses > 1000000 then 2.0
    If losses > 500000 then 1.5
    If losses > 250000 then 1.0
    If losses > 100000 then 0.85
    Else 0.75`


In [122]:
def clrt (exposure, losses):
    technical_preminum = 0.0
    if exposure < 100:
        if losses > 1000000:
            technical_preminum = 1.8
        if losses > 500000:
            technical_preminum = 1.4
        if losses > 250000:
            technical_preminum = 1.0
        if losses > 100000:
            technical_preminum = 1.9
        else:
            technical_preminum = 0.8
    else:
        if losses > 1000000:
            technical_preminum = 2.0
        if losses > 500000:
            technical_preminum = 1.5
        if losses > 250000:
            technical_preminum = 1.0
        if losses > 100000:
            technical_preminum = 0.85
        else:
            technical_preminum = 0.75
    return technical_preminum

In [123]:
clrt_df = test_df.copy()
clrt_df['experience'] = 0.0
clrt_df['clrt'] = 0.0
for i, row in clrt_df.iterrows():
    clrt_df.at[i , 'experience'] = experience_rater(row['experience_rated_manual_premium'], row['incurred_loss_and_alae'])
    clrt_df.at[i , 'clrt'] = clrt(row['n_power_units'], row['incurred_loss_and_alae'])
clrt_df['technical_premium'] = clrt_df['clrt'] * clrt_df['experience'] * clrt_df['experience_rated_manual_premium']
clrt_df.head()

,const,pol_eff_year,risk_state,naics3,n_power_units,prior_claim_freq_3yr,dnb_credit_score,easi_snowfall,experience_rated_manual_premium,incurred_loss_and_alae,experience,clrt,technical_premium
0,1.0,2010,AK,722.0,24,0.0,0.0,70.0,478.555077,0.000000,0.8,0.8,306.275249
8,1.0,2010,AR,453.0,1,0.0,2.0,5.0,429.164457,0.000000,0.8,0.8,274.665253
9,1.0,2010,AR,423.0,4,0.0,1.0,7.0,1234.672690,0.000000,0.8,0.8,790.190522
18,1.0,2010,AZ,112.0,17,0.0,3.0,0.0,3274.963002,423.294916,0.8,0.8,2095.976321
19,1.0,2010,AZ,112.0,32,0.0,3.0,0.0,2978.807420,0.000000,0.8,0.8,1906.436749


### 4. Discretionary Pricing Guidance (tool_name = NBPT, file_type = Excel, model_type = GLM)
- Relies on (1) and (2)
- Sample data and schema provided
- Please build a GLM with the following model specification
    - link = “log”
    - family = “Tweedie”
    - weight = manual_premium
    - target = manual_loss_ratio (incurred_loss_and_alae / manual_premium)
    - features = [
        - policy_year
        - risk_state
        - n_power_units
        - prior_claim_freq_3yr
        - easi_snowfall
        - dnb_credit_score
    ]
- Output is technical premium
    - GLM prediction * Experience Mod * Manual Premium * (1 + Indication) = Technical Premium
    - Indications are provided in a sample file
        - [sample_data].[risk_state] = [indication].[risk_state]
        - [sample_data].[coverage_type] = [indication].[coverage_type]
        - Join on these keys to obtain the Indication

In [64]:
glm_model = sm.GLM(train_y, train_x, family=sm.families.Tweedie(link=sm.families.links.log()), var_weights=np.asarray(train_x["experience_rated_manual_premium"]))
glm_results = glm_model.fit()
print(glm_results.summary())

/home/jovyan/.local/lib/python3.6/site-packages/statsmodels/genmod/families/family.py:1444: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jovyan/.local/lib/python3.6/site-packages/statsmodels/genmod/families/family.py:1444: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))


                 Generalized Linear Model Regression Results                  
Dep. Variable:      manual_loss_ratio   No. Observations:               103414
Model:                            GLM   Df Residuals:                   103405
Model Family:                 Tweedie   Df Model:                            8
Link Function:                    log   Scale:                          6763.9
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 17 Dec 2021   Deviance:                   2.3890e+08
Time:                        20:33:14   Pearson chi2:                 6.99e+08
No. Iterations:                    12                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [121]:
nbpt_df = test_df.copy()
nbpt_df['glm'] = glm_results.predict(test_df.drop(['risk_state'], axis=1))
nbpt_df['indication'] = nbpt_df.join(indication.set_index('risk_state'), on='risk_state')["state_sold_indication"] + 1
nbpt_df['experience'] = 0.0
for i, row in nbpt_df.iterrows():
    nbpt_df.at[i , 'experience'] = experience_rater(row['experience_rated_manual_premium'], row['incurred_loss_and_alae'])
nbpt_df['technical_premium'] = nbpt_df['glm'] * nbpt_df['experience'] * nbpt_df['experience_rated_manual_premium'] * nbpt_df['indication']
nbpt_df.head()

,const,pol_eff_year,risk_state,naics3,n_power_units,prior_claim_freq_3yr,dnb_credit_score,easi_snowfall,experience_rated_manual_premium,incurred_loss_and_alae,glm,indication,experience,technical_premium
0,1.0,2010,AK,722.0,24,0.0,0.0,70.0,478.555077,0.000000,0.227651,0.970010,0.8,84.540979
8,1.0,2010,AR,453.0,1,0.0,2.0,5.0,429.164457,0.000000,0.369882,1.003008,0.8,127.374334
9,1.0,2010,AR,423.0,4,0.0,1.0,7.0,1234.672690,0.000000,0.354746,1.003008,0.8,351.450504
18,1.0,2010,AZ,112.0,17,0.0,3.0,0.0,3274.963002,423.294916,0.385883,1.234135,0.8,1247.713084
19,1.0,2010,AZ,112.0,32,0.0,3.0,0.0,2978.807420,0.000000,0.375839,1.234135,0.8,1105.342987


### 5. If the CLRT is used, (3) and (4) are credibility weighted for final technical premium
- z * Technical Premium from (3) + (1-z) * Technical Premium from (4)
- Assume z = .25

In [125]:
z = 0.25
technical_premium = z * clrt_df['technical_premium'] + (1 - z) * nbpt_df['technical_premium']
technical_premium.head()

0      139.974546
8      164.197064
9      461.135508
18    1459.778893
19    1305.616427
Name: technical_premium, dtype: float64

### 6. Deploy the Technical Premium endpoint and show us a sample request/response using provided data